In [ ]:
import sys
sys.path.append("../")

from CurveDataFetcher import CurveDataFetcher 
from HedgeHog import beta_estimates, dv01_neutral_curve_hegde_ratio, dv01_neutral_butterfly_hegde_ratio
from CurveInterpolator import GeneralCurveInterpolator

from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import statsmodels.api as sm

from utils.pca_utils import calc_pca_loadings_matrix
from utils.ust_viz import (
    plot_usts, 
    plot_timeseries, 
)
from utils.regression_utils import (
    plot_residuals_timeseries,
    run_basic_linear_regression_df, 
    run_basic_linear_regression,
    run_multiple_linear_regression_df
)

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import matplotlib.pylab as pylab
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)
sns.set(style="whitegrid", palette="dark")
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Curvy-CUSIPs\notebooks\../.env
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

In [150]:
tenors = ["CT3M", "CT6M", "CT1", "CT2", "CT3", "CT5", "CT7", "CT10", "CT20", "CT30"]
bid_ct_yields_df = curve_data_fetcher.fedinvest_data_fetcher.get_historical_ct_yields(start_date=datetime(2023, 11, 1), end_date=datetime(2024, 10, 31), use_offer_side=True)
offer_ct_yields_df = curve_data_fetcher.fedinvest_data_fetcher.get_historical_ct_yields(start_date=datetime(2023, 11, 1), end_date=datetime(2024, 10, 31), use_bid_side=True)
bid_offer_spread_ct_yields_df: pd.DataFrame = bid_ct_yields_df.set_index("Date") - offer_ct_yields_df.set_index("Date")
display(bid_offer_spread_ct_yields_df)

# EOD FedInvest Marking s
ct_yields_df: pd.DataFrame = curve_data_fetcher.fedinvest_data_fetcher.get_historical_ct_yields(start_date=datetime(2023, 11, 1), end_date=datetime(2024, 10, 31)) 
ct_yields_df

,CT2M,CT3M,CT6M,CT1,CT2,CT3,CT5,CT7,CT10,CT20,CT30
Date,,,,,,,,,,,
2023-11-01,NaN,-0.005394,-0.005394,-0.011005,0.000000,0.000000,0.000000,-0.002650,-0.002110,-0.001354,-0.001115
2023-11-02,NaN,-0.010786,-0.005366,-0.005520,0.000000,-0.005757,0.000000,0.000000,-0.002083,-0.001323,-0.001080
2023-11-03,NaN,-0.005492,-0.005427,-0.005545,0.000000,0.000000,0.000000,0.000000,-0.002059,-0.001297,-0.001051
2023-11-06,NaN,-0.005388,-0.005362,-0.005511,0.000000,-0.005763,-0.003517,-0.002624,0.000000,-0.001323,0.000000
2023-11-07,NaN,-0.005329,-0.010725,-0.005512,0.000000,-0.005768,0.000000,-0.002618,0.000000,-0.001306,-0.001062
...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25,NaN,-0.010942,-0.005387,-0.005443,-0.008582,-0.005653,0.000000,-0.002660,0.000000,-0.001251,0.000000
2024-10-28,NaN,0.000000,-0.005488,-0.005494,0.000000,0.000000,-0.003583,0.000000,-0.001998,-0.001256,-0.000988
2024-10-29,NaN,-0.011595,-0.005591,-0.005541,0.000000,-0.005652,-0.003570,-0.002653,0.000000,-0.001246,0.000000


,Date,CT2M,CT3M,CT6M,CT1,CT2,CT3,CT5,CT7,CT10,CT20,CT30
0,2023-11-01,NaN,5.561712,5.534145,5.405577,4.949711,4.761043,4.669336,4.752907,4.764730,5.136924,4.957943
1,2023-11-02,NaN,5.550600,5.495336,5.383786,4.982799,4.784095,4.648023,4.684300,4.668620,5.003251,4.815454
2,2023-11-03,NaN,5.527971,5.451418,5.294875,4.831896,4.623056,4.485916,4.547775,4.561250,4.937633,4.761956
3,2023-11-06,NaN,5.527555,5.461670,5.338526,4.931800,4.726586,4.605023,4.657556,4.660988,5.014037,4.832860
4,2023-11-07,NaN,5.458567,5.461257,5.338215,4.914796,4.691987,4.534499,4.584032,4.573958,4.916826,4.740761
...,...,...,...,...,...,...,...,...,...,...,...,...
246,2024-10-25,NaN,4.692279,4.527057,4.311949,4.112120,4.054808,4.064476,4.153506,4.243296,4.596421,4.511968
247,2024-10-28,NaN,4.894568,4.643728,4.355614,4.078486,4.032231,4.050390,4.142991,4.235377,4.588897,4.511976
248,2024-10-29,NaN,5.047442,4.718547,4.361846,4.027588,3.953032,3.979041,4.073929,4.179627,4.546272,4.486353
249,2024-10-30,NaN,4.639681,4.494619,4.289765,4.166252,4.134693,4.144421,4.207522,4.279613,4.606625,4.492263


In [152]:
pca_results = calc_pca_loadings_matrix(
    df=ct_yields_df.set_index("Date"), 
    cols=["CT3M", "CT6M", "CT1", "CT2", "CT3", "CT5", "CT7", "CT10", "CT20", "CT30"], 
    run_on_level_changes=True,
    scale_loadings=True 
)
pca_results

{'pca': PCA(),
 'loading_matrix':            PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
 CT3M  0.024168  0.069391  0.015465 -0.009201 -0.004158 -0.000155  0.000107   
 CT6M  0.029161  0.048306  0.003424  0.007191  0.009908  0.001843  0.000207   
 CT1   0.054063  0.027414 -0.014080  0.011070 -0.006147 -0.004312 -0.000930   
 CT2   0.075453  0.003464 -0.024247 -0.001087 -0.002780  0.007502 -0.001228   
 CT3   0.079609 -0.003023 -0.017718 -0.003852  0.001662 -0.002460  0.005420   
 CT5   0.081416 -0.009353 -0.005346 -0.004743  0.002781 -0.001681 -0.001992   
 CT7   0.078794 -0.012300  0.003891 -0.002693  0.002217 -0.002523 -0.002876   
 CT10  0.072477 -0.013837  0.011525 -0.000969  0.001174 -0.001682 -0.001131   
 CT20  0.065026 -0.015192  0.020077  0.002950 -0.001228  0.002650  0.001105   
 CT30  0.060439 -0.014348  0.025074  0.004626 -0.003061  0.001624  0.001683   
 
            PC8       PC9      PC10  
 CT3M  0.000058 -0.000088 -0.000008  
 CT6M -0.000214  0.0

In [41]:
date = datetime(2024, 10, 31)
curve_set_df: pd.DataFrame = curve_data_fetcher.fetch_historical_curve_sets(start_date=date, calc_free_float=True)[date]

otr_df: pd.DataFrame = curve_set_df[curve_set_df["rank"] == 1].sort_values(by="time_to_maturity")

curve_set_filtered_df = curve_set_df[
    (curve_set_df["rank"] != 0)
    & (curve_set_df["rank"] != 1)
    & (curve_set_df["rank"] != 2)
    & (curve_set_df["rank"] != 3)
]
curve_set_filtered_df = curve_set_filtered_df[
    curve_set_filtered_df["security_type"] != "Bill"
]
curve_set_filtered_df = curve_set_filtered_df[
    curve_set_filtered_df["free_float"] > 5000
]
curve_set_filtered_df = curve_set_filtered_df[
    curve_set_filtered_df["time_to_maturity"] > 30 / 360
]
curve_set_filtered_df = curve_set_filtered_df.sort_values(by=["time_to_maturity"])

quote_type = "eod"
filtered_fitted_interpolator = GeneralCurveInterpolator(
    x=curve_set_filtered_df["time_to_maturity"].to_numpy(),
    y=curve_set_filtered_df[f"{quote_type}_yield"].to_numpy(),
)
fitted_bspline = filtered_fitted_interpolator.b_spline_with_knots_interpolation(
    knots=[0.5, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 15, 20, 25],
    k=3,
    return_func=True,
)

AGGREGATING CURVE SET DFs: 100%|██████████| 4/4 [00:00<00:00, 255.44it/s]


In [144]:
ct2_row = otr_df[otr_df["original_security_term"] == "2-Year"]
ct5_row = otr_df[otr_df["original_security_term"] == "5-Year"]
ct10_row = otr_df[otr_df["original_security_term"] == "10-Year"]

curve_beta_estimates_results = beta_estimates(
    df=ct_yields_df,
    x_cols=["CT2"],
    y_col="CT10",
    run_on_level_changes=True,
    # pc_scores_df=pca_results["pc_scores_matrix"],
    loadings_df=pca_results["loading_matrix"],
    x_errs=bid_offer_spread_ct_yields_df["CT2"].to_numpy(),
    y_errs=bid_offer_spread_ct_yields_df["CT10"].to_numpy(),
)

c:\Users\chris\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [2]:
dv01_neutral_curve_hegde_ratio(
    as_of_date=date,
    front_leg_bond_row=ct2_row,
    back_leg_bond_row=ct10_row,
    curve_data_fetcher=curve_data_fetcher,
    scipy_interp_curve=fitted_bspline,
    repo_rate=4.83,
    yvx_beta_adjustment=curve_beta_estimates_results["betas"]["ols"],
    total_trade_par_amount=10_000_000,
)

NameError: name 'dv01_neutral_curve_hegde_ratio' is not defined

In [3]:
ct2_row = otr_df[otr_df["original_security_term"] == "2-Year"]
ct5_row = otr_df[otr_df["original_security_term"] == "5-Year"]
ct10_row = otr_df[otr_df["original_security_term"] == "10-Year"]

fly_beta_estimates_results = beta_estimates(
    df=ct_yields_df,
    x_cols=["CT2", "CT10"],
    y_col="CT5",
    run_on_level_changes=True,
    pc_scores_df=pca_results["pc_scores_matrix"],
    loadings_df=pca_results["loading_matrix"],
)
fly_beta_estimates_results["betas"]

NameError: name 'otr_df' is not defined

In [4]:
dv01_neutral_butterfly_hegde_ratio(
    as_of_date=date,
    front_wing_bond_row=ct2_row,
    belly_bond_row=ct5_row,
    back_wing_bond_row=ct10_row,
    curve_data_fetcher=curve_data_fetcher,
    scipy_interp_curve=fitted_bspline,
    repo_rate=4.83,
    yvx_front_wing_beta_adjustment=fly_beta_estimates_results["betas"]["pc1"][1],
    yvx_back_wing_beta_adjustment=fly_beta_estimates_results["betas"]["pc1"][0],
    total_trade_par_amount=10_000_000,
)

NameError: name 'dv01_neutral_butterfly_hegde_ratio' is not defined